In [12]:
import sys
sys.path.insert(1, '../../model/')
sys.path.append('/nlp/data/yueyang/prototypicality/semantic-norms-main')
import eval
import pickle
from tqdm import tqdm
from nltk.corpus import wordnet as wn
import openai
openai.api_key = "<OPENAI-KEY>"

In [13]:
DATASET = "concept_properties" # "feature_norms"

In [16]:
noun2prop = pickle.load(open(f"../../data/datasets/{DATASET}/noun2property/noun2prop.p", "rb"))

In [17]:
candidate_adjs = []
for noun, props in noun2prop.items():
    candidate_adjs += props
candidate_adjs = list(set(candidate_adjs))

In [76]:
noun2predicts = {}
for noun in tqdm(noun2prop):
    response = openai.Completion.create(
        engine="text-davinci-001",
        prompt="Use ten adjectives to describe the properties of kiwi:\n1. tart\n2. acidic\n3. sweet\n4. juicy\n5. smooth\n6. fuzzy\n7. green\n8. brown\n9. small\n10. fuzzy\nUse ten adjectives to describe the properties of {}:\n".format(noun),
        temperature=0,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    props = []
    for raw_prop in dict(dict(response)['choices'][0])['text'].split('\n'):
        if len(raw_prop) >= 5 and '.' in raw_prop:
            props.append(raw_prop.split(' ')[-1].lower())
    noun2predicts[noun] = props

100%|██████████| 509/509 [06:53<00:00,  1.23it/s]


In [ ]:
acc_1 = eval.evaluate_acc(noun2predicts, noun2prop, 1, True)
acc_2 = eval.evaluate_acc(noun2predicts, noun2prop, 2, True)
acc_3 = eval.evaluate_acc(noun2predicts, noun2prop, 3, True)
acc_5 = eval.evaluate_acc(noun2predicts, noun2prop, 5, True)
r_5 = eval.evaluate_recall(noun2predicts, noun2prop, 5, True)
r_10 = eval.evaluate_recall(noun2predicts, noun2prop, 10, True)